In [3]:
# CE Reranker on CPU: NLI DeBERTa v3 base with asymmetric CPC-on-anchor prompt
# - Produces raw CE scores and per-fold isotonic calibrated scores
# - Overwrites oof_ce_stsb.csv/submission_ce_stsb.csv with RAW reranker scores for stacker consumption
# - Writes oof_ce_l12.csv/submission_ce_l12.csv with ce_iso for optional usage in LGBM stacker
import time, numpy as np, pandas as pd
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sentence_transformers import CrossEncoder

SEED = 42
np.random.seed(SEED)

# CPC mapping (4->3->2 title; RAW code fallback), truncate to 10 words
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying; atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return code  # RAW fallback

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_pairs_asym(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a_side = (cpc_10w + ' [SEP] ' + df['anchor'].astype(str)).tolist()
    b_side = df['target'].astype(str).tolist()
    return list(zip(a_side, b_side))

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

train_pairs = build_pairs_asym(train)
test_pairs = build_pairs_asym(test)

# Public reranker backbone suggested by experts; CPU-friendly
model_name = 'cross-encoder/nli-deberta-v3-base'
ce = CrossEncoder(model_name, device='cpu', max_length=256, tokenizer_args={'use_fast': False})

def predict_entailment_prob(ce: CrossEncoder, pairs, batch_size=64):
    # Get softmax probabilities and select the 'entailment' index using id2label when available
    probs = np.asarray(ce.predict(pairs, batch_size=batch_size, show_progress_bar=True, apply_softmax=True))
    ent_idx = None
    try:
        id2label = getattr(ce.model.config, 'id2label', None)
        if isinstance(id2label, dict) and len(id2label) >= 2:
            for k, v in id2label.items():
                name = str(v).lower()
                if 'entail' in name:
                    ent_idx = int(k) if isinstance(k, (int, np.integer)) or str(k).isdigit() else None
                    break
    except Exception:
        pass
    if ent_idx is None:
        # Fallback: common mapping for NLI is 0:contradiction,1:neutral,2:entailment
        ent_idx = 2 if probs.shape[1] == 3 else probs.shape[1]-1
    return probs[:, ent_idx].astype(np.float32)

print('Scoring train pairs (nli-deberta-v3-base, context on anchor)...', flush=True)
train_scores_raw = predict_entailment_prob(ce, train_pairs, batch_size=64)
print('Scoring test pairs (nli-deberta-v3-base, context on anchor)...', flush=True)
test_scores_raw = predict_entailment_prob(ce, test_pairs, batch_size=64)

# Per-fold isotonic vs linear calibration for diagnostics
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros_like(y, dtype=np.float32)
oof_lin = np.zeros_like(y, dtype=np.float32)
test_iso_f, test_lin_f = [], []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(train_scores_raw[va_idx], y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(train_scores_raw[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(train_scores_raw[va_idx]).astype(np.float32)
    test_iso_f.append(iso.predict(test_scores_raw).astype(np.float32))
    lr = LinearRegression()
    lr.fit(train_scores_raw[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(train_scores_raw[va_idx].reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(test_scores_raw.reshape(-1,1)).astype(np.float32))
    print(f'NLI-CE Fold {int(f)}: raw={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

p_raw = pearsonr(train_scores_raw, y)[0]
p_iso = pearsonr(oof_iso, y)[0]
p_lin = pearsonr(oof_lin, y)[0]
print('NLI-CE OOF Pearson: raw=', round(float(p_raw),6), ' iso=', round(float(p_iso),6), ' lin=', round(float(p_lin),6))

# Save dedicated artifacts
pd.DataFrame({'id': train['id'], 'oof_raw': train_scores_raw, 'oof_iso': oof_iso}).to_csv('oof_ce_bge_rerank.csv', index=False)
pd.DataFrame({'id': test['id'], 'score_raw': test_scores_raw, 'score_iso': np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)}).to_csv('submission_ce_bge_rerank.csv', index=False)

# Overwrite CE artifacts used by stackers:
pd.DataFrame({'id': train['id'], 'oof': train_scores_raw}).to_csv('oof_ce_stsb.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': test_scores_raw}).to_csv('submission_ce_stsb.csv', index=False)
pd.DataFrame({'id': train['id'], 'oof_raw': train_scores_raw, 'oof_iso': oof_iso}).to_csv('oof_ce_l12.csv', index=False)
pd.DataFrame({'id': test['id'], 'score_raw': test_scores_raw, 'score_iso': np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)}).to_csv('submission_ce_l12.csv', index=False)

print('Saved NLI DeBERTa CE artifacts and overwrote CE files for stacker. Elapsed', round((time.time()-t0)/60,2), 'min')

Scoring train pairs (nli-deberta-v3-base, context on anchor)...


Batches:   0%|          | 0/513 [00:00<?, ?it/s]

Batches:   0%|          | 1/513 [00:00<04:14,  2.01it/s]

Batches:   0%|          | 2/513 [00:00<03:22,  2.52it/s]

Batches:   1%|          | 3/513 [00:01<03:09,  2.69it/s]

...

NLI-CE Fold 1: raw=0.424813 | iso=0.442479 | lin=0.424813


NLI-CE Fold 2: raw=0.418987 | iso=0.434908 | lin=0.418987


NLI-CE Fold 3: raw=0.421060 | iso=0.446963 | lin=0.421060


NLI-CE Fold 4: raw=0.428902 | iso=0.445477 | lin=0.428902


NLI-CE OOF Pearson: raw= 0.426387  iso= 0.445772  lin= 0.42618
Saved NLI DeBERTa CE artifacts and overwrote CE files for stacker. Elapsed 3.3 min


In [4]:
# BM25 features (CPU, no leakage): per-fold IDF on train-only; compute bm25_ab (anchor->target) and bm25_ba (target->anchor)
# Test features are averaged over folds (fit IDF on train!=fold, score all test rows).
import time, re, math, numpy as np, pandas as pd
from collections import Counter
from tqdm.auto import tqdm

SEED = 42
np.random.seed(SEED)

def tokenize_words(s: str):
    return re.findall(r"\w+", str(s).lower())

def build_df(tokens_list):
    df = Counter()
    for toks in tokens_list:
        df.update(set(toks))  # document frequency counts presence once per doc
    return df

def bm25_idf(df_counter, N):
    # idf(t) = log((N - df + 0.5)/(df + 0.5) + 1)
    idf = {}
    for t, df in df_counter.items():
        idf[t] = math.log((N - df + 0.5)/(df + 0.5) + 1.0)
    return idf

def bm25_score(query_tokens, doc_tokens, idf, avgdl, k1=1.5, b=0.75):
    if not query_tokens or not doc_tokens:
        return 0.0
    f = Counter(doc_tokens)
    dl = len(doc_tokens)
    score = 0.0
    # Use unique terms from query to avoid overweighting repeated query terms (common practice in IR scoring)
    for term in set(query_tokens):
        if term not in idf:
            continue
        tf = f.get(term, 0)
        if tf == 0:
            continue
        denom = tf + k1 * (1.0 - b + b * (dl / (avgdl + 1e-12)))
        score += idf[term] * (tf * (k1 + 1.0)) / (denom + 1e-12)
    return float(score)

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Pre-tokenize
A_tr_tok = [tokenize_words(x) for x in train['anchor'].astype(str).tolist()]
B_tr_tok = [tokenize_words(x) for x in train['target'].astype(str).tolist()]
A_te_tok = [tokenize_words(x) for x in test['anchor'].astype(str).tolist()]
B_te_tok = [tokenize_words(x) for x in test['target'].astype(str).tolist()]

fold_arr = train['fold'].values.astype(int)
n_tr = len(train); n_te = len(test)

bm25_ab_oof = np.zeros(n_tr, dtype=np.float32)
bm25_ba_oof = np.zeros(n_tr, dtype=np.float32)
bm25_ab_te_folds = []
bm25_ba_te_folds = []

for f in sorted(np.unique(fold_arr)):
    f0 = time.time()
    tr_idx = np.where(fold_arr != f)[0]
    va_idx = np.where(fold_arr == f)[0]
    # Fit IDF on train-only for targets (for A->B) and anchors (for B->A)
    corpus_B = [B_tr_tok[i] for i in tr_idx]
    corpus_A = [A_tr_tok[i] for i in tr_idx]
    N_B = len(corpus_B); N_A = len(corpus_A)
    df_B = build_df(corpus_B); idf_B = bm25_idf(df_B, N_B)
    df_A = build_df(corpus_A); idf_A = bm25_idf(df_A, N_A)
    avgdl_B = float(np.mean([len(t) for t in corpus_B]) if corpus_B else 0.0)
    avgdl_A = float(np.mean([len(t) for t in corpus_A]) if corpus_A else 0.0)

    # Compute OOF for this fold
    for i in va_idx:
        # bm25_ab: query = anchor, doc = target
        bm25_ab_oof[i] = bm25_score(A_tr_tok[i], B_tr_tok[i], idf_B, avgdl_B)
        # bm25_ba: query = target, doc = anchor
        bm25_ba_oof[i] = bm25_score(B_tr_tok[i], A_tr_tok[i], idf_A, avgdl_A)

    # Compute test features for this fold
    te_ab = np.zeros(n_te, dtype=np.float32)
    te_ba = np.zeros(n_te, dtype=np.float32)
    for j in range(n_te):
        te_ab[j] = bm25_score(A_te_tok[j], B_te_tok[j], idf_B, avgdl_B)
        te_ba[j] = bm25_score(B_te_tok[j], A_te_tok[j], idf_A, avgdl_A)
    bm25_ab_te_folds.append(te_ab)
    bm25_ba_te_folds.append(te_ba)
    print(f'BM25 fold {int(f)} done in {time.time()-f0:.1f}s', flush=True)

# Aggregate test across folds (mean)
bm25_ab_te = np.mean(np.vstack(bm25_ab_te_folds), axis=0).astype(np.float32)
bm25_ba_te = np.mean(np.vstack(bm25_ba_te_folds), axis=0).astype(np.float32)

# Save artifacts
pd.DataFrame({'id': train['id'], 'bm25_ab': bm25_ab_oof, 'bm25_ba': bm25_ba_oof}).to_csv('oof_bm25.csv', index=False)
pd.DataFrame({'id': test['id'], 'bm25_ab': bm25_ab_te, 'bm25_ba': bm25_ba_te}).to_csv('bm25_test.csv', index=False)
print('Saved oof_bm25.csv and bm25_test.csv; elapsed', round((time.time()-t0)/60,2), 'min')

BM25 fold 0 done in 0.1s


BM25 fold 1 done in 0.1s


BM25 fold 2 done in 0.1s


BM25 fold 3 done in 0.1s


BM25 fold 4 done in 0.1s


Saved oof_bm25.csv and bm25_test.csv; elapsed 0.01 min
